### IMPORT LIBRARIES


In [1]:
import cv2
import numpy as np


## TASK 3


### Video Capture Setup

### Run this code and it will open camera and record video"


In [3]:

cap = cv2.VideoCapture(0) 

if not cap.isOpened():
    print("Cannot open camera.")
else:
 
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))  

    while True:

        ret, frame = cap.read()

  
        if not ret:
            print("Cannot capture frame from the camera.")
            break

       
        out.write(frame)

    
        cv2.imshow('Camera Feed', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()



### Real-Time Video Processing Loop

### Run this code it will display video 

In [4]:

video_path = 'output.avi'  

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Cannot open video file.")
else:
    while True:
        ret, frame = cap.read()

        if not ret:
            print("End of video.")
            break
     
        cv2.imshow('Video Playback', frame)
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



End of video.


### For each captured frame


In [12]:
video_path = 'output.avi'  
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Cannot open video file.")
else:
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('processed_video.avi', fourcc, 20.0, (frame_width, frame_height))
    
    while True:        
        ret, frame = cap.read()        
        if not ret:
            print("End of video.")
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        
        # Apply histogram equalization for contrast enhancement.
        equ = cv2.equalizeHist(gray_frame)
        
        # Convert the frame to a colored heatmap using the "jet" colormap.
        heatmap = cv2.applyColorMap(equ, cv2.COLORMAP_JET)

        # Apply color balance adjustment to correct any color casts.
        alpha = 1.5  
        beta = 1.2   
        color_balanced = heatmap.copy()
        color_balanced[:, :, 0] = np.clip(heatmap[:, :, 0] * alpha, 0, 255).astype(np.uint8)
        color_balanced[:, :, 2] = np.clip(heatmap[:, :, 2] * beta, 0, 255).astype(np.uint8)
        
        # Apply a logarithmic transformation to enhance visibility in dark regions.
        log_transformed = np.log1p(color_balanced)
        
        # Apply a power-law transformation for fine-tuning contrast.
        gamma = 0.7
        power_law = np.power(log_transformed, gamma)

        power_law_visualized = (255 * (power_law / np.max(power_law))).astype(np.uint8)
        
        resize_width = 320

        frame_resized = cv2.resize(frame, (resize_width, frame_height)) # Display the original live video stream.
        gray_frame_resized = cv2.cvtColor(cv2.resize(gray_frame, (resize_width, frame_height)), cv2.COLOR_GRAY2BGR)
        heatmap_resized = cv2.resize(heatmap, (resize_width, frame_height))
        color_balanced_resized = cv2.resize(color_balanced, (resize_width, frame_height))
        power_law_resized = cv2.resize(power_law_visualized, (resize_width, frame_height))

        # Display individual frames with each operation applied side by side for comparison.
        side_by_side = np.hstack((frame_resized, gray_frame_resized, heatmap_resized, color_balanced_resized, power_law_resized))
        cv2.imshow('Frames Comparison', side_by_side)

        out.write(power_law_visualized)  

       
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    
    cap.release()
    out.release()
    cv2.destroyAllWindows()


End of video.
